In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider
from bokeh.models import HoverTool
from bokeh.models import WMTSTileSource
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource

output_notebook()

Loading BokehJS ...

In [2]:
import pandas as pd
import numpy as np

def wgs84_webmercator(df, lon="long", lat="lat", lon1="bellong", lat1="bellat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    
    df["x0"] = df[lon] * (k * np.pi/180.0)
    df["y0"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
        
    df["x1"] = df[lon1] * (k * np.pi/180.0)
    df["y1"] = np.log(np.tan((90 + df[lat1]) * np.pi/360.0)) * k
    
    return df

In [3]:
hour = 16
direction = 'arrival'

In [4]:
path = 'data/data.csv'
df = pd.read_csv(path)
df = df.fillna(0)

wgs84_webmercator(df).head()
m = df['6'].max()

df = df.loc[df['direction'] == direction]
df['PHV'] = df['5'] + df['6'] + df['7']


def setwidth (x, h):
    h = str (h)
    x['width'] = x[h].apply(lambda x: x/m*20 + 1)
    x = x.dropna(subset=['width'])
    return x

if direction == 'arrival':
    colour = 'green'
else: colour = 'red'

df = setwidth (df, hour)

df.to_csv('output/BellvilleTaxiOD.csv')
df.head()

,direction,area,long,lat,bellat,bellong,0,5,6,7,...,16,17,18,19,x0,y0,x1,y1,PHV,width
0,arrival,Atlantis,18.486956,-33.506266,-33.905804,18.629444,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,2.057958e+06,-3.962697e+06,2.073820e+06,-4.016161e+06,0.0,1.000000
1,arrival,Belhar,18.623500,-33.943635,-33.905804,18.629444,0.0,25.0,121.0,200.0,...,96.0,65.0,31.0,0.0,2.073159e+06,-4.021236e+06,2.073820e+06,-4.016161e+06,346.0,1.222325
2,arrival,Bellville South,18.645026,-33.922112,-33.905804,18.629444,0.0,43.0,55.0,55.0,...,58.0,37.0,17.0,0.0,2.075555e+06,-4.018348e+06,2.073820e+06,-4.016161e+06,153.0,1.134321
3,arrival,Blackheath,18.697731,-33.966985,-33.905804,18.629444,0.0,0.0,45.0,0.0,...,28.0,21.0,0.0,0.0,2.081422e+06,-4.024370e+06,2.073820e+06,-4.016161e+06,45.0,1.064845
4,arrival,Bloekombos,18.749701,-33.844489,-33.905804,18.629444,0.0,6.0,30.0,42.0,...,16.0,37.0,0.0,0.0,2.087207e+06,-4.007940e+06,2.073820e+06,-4.016161e+06,78.0,1.037054


In [5]:
p = figure(plot_width=700, plot_height=500, tools='pan,wheel_zoom,box_zoom,save,reset')

tile_provider = get_provider(CARTODBPOSITRON)
p.add_tile(tile_provider)

source = ColumnDataSource(df)

p.title.text = 'Total MBT %ss at Bellville by origin - Hour %s' % (direction, hour)

p.segment(x0='x0', y0='y0', x1='x1',
          y1='y1', color=colour, line_width='width', alpha=0.5, 
         source=source)

p.circle(x=2073820, y=-4016161, fill_color="#669bbc", size=10)

p.circle(x='x0', y='y0', 
         source=source, 
         size=7, color='orange', alpha=0.5)

hover = HoverTool()
hover.tooltips=[
    ('Area', '@area'),
    ('Direction', '@direction'),
    ('Peak Hour Volume', '@%s' % (hour)),
    ('Peak Period Volume', '@PHV')
]

p.add_tools(hover)

show(p)

In [13]:
output_file('D:/002_OtherWork/05 - Bellville/test.html')